In [1]:
# Denpendancies
import os
import numpy as np
import pandas as pd
from dLoader import DataLoader

In [2]:
# Load tickers from Database
def get_tickers(database='Database'):
    db = os.path.join(os.getcwd(), database)
    if os.path.exists(db):
        tickers = [ld.split('.')[0] for ld in os.listdir(db)]
        return sorted(tickers)
    print('Database could not be found.')

def batching(data, period):
    batch = len(data) // period
    max_length = batch * period
    return np.array(data)[-max_length:].reshape(-1, period)

def regroup(data, period):
    base = batching(data['Open'], period)[:, 0]
    high = batching(data['High'], period).max(1)
    low = batching(data['Low'], period).min(1)
    close = batching(data['Close'], period)[:, -1]
    return pd.DataFrame(np.stack([base, high, low, close], axis=1), 
                        columns=['Open', 'High', 'Low', 'Close'])

In [3]:
ticker = DataLoader('AAPL').get_data('2018-01-01', '2020-12-31')

In [9]:
# Calculate Peps
hl = (ticker['High'].shift(-1) / ticker['Low'] - 1).dropna()
hc = (ticker['High'].shift(-1) / ticker['Close'] - 1).dropna()
cc = (ticker['Close'].shift(-1) / ticker['Close'] - 1).dropna()
oc = (ticker['Open'].shift(-1) / ticker['Close'] - 1).dropna()

def print_report(g):
    m = g > 0
    plen = len(g[m]) / len(g)
    pM = g[m].mean()
    nlen = len(g[~m]) / len(g)
    nM = g[~m].mean()
    _rang = g[m].mean() + np.abs(g[~m].mean())
    pep = _rang / (7.5 / .5)
    print('{:.2f}::{:.4f} | {:.2f}::{:.4f} | {:.4f}'.format(
        plen, pM, nlen, nM, pep))
    return {'P PCT': plen, 
            'P Mean': pM, 
            'N PCT': nlen, 
            'N Mean': nM, 
            'Pep': pep}

def get_pep(g, time=(7.5 / .5)):
    m = g > 0
    _rang = g[m].mean() + np.abs(g[~m].mean())
    return _rang / time

oc_dic = print_report(oc)
cc_dic = print_report(cc)
hc_dic = print_report(hc)
hl_dic = print_report(hl)

0.57::0.0084 | 0.43::-0.0097 | 0.0012
0.54::0.0154 | 0.46::-0.0146 | 0.0020
0.86::0.0160 | 0.14::-0.0099 | 0.0017
0.94::0.0264 | 0.06::-0.0079 | 0.0023


In [5]:
pep = hc_dic['Pep']

In [6]:
def back_test(ticker, pep):
    holding = False
    max_shares = 1000
    #
    base = 0
    shares = 0
    #
    capital = 1000
    base_capital = capital
    #
    sell = 1 + (3 * pep)
    stop = 1 + (-1 * pep)
    for date, prices in ticker.iterrows():
        if not holding:
            base = prices.Close
            shares = capital // base
            shares = max_shares if shares > max_shares else shares
            #
            min_sell = base * sell
            stop_price = base * stop
            #
            holding = True
        else:
            if prices.Open < stop_price:
                gain = prices.Open - base
            elif prices.High > min_sell:
                gain = (prices.High * stop) - base
            else:
                gain = prices.Close - base

            capital += np.round(gain * shares, 2)
            # 
            base = 0
            shares = 0
            #
            holding = False

    return capital / base_capital - 1

In [7]:
back_test(ticker, pep)

9.701970000000003

In [11]:
tickers = get_tickers()

In [28]:
dic = {}
for ticker in tickers:
    data = DataLoader(ticker).get_data('2018-01-01', '2020-12-31')
    hc = (data['High'].shift(-1) / data['Close'] - 1).dropna()
    pep = get_pep(hc)
    above_pep = (hc > pep).mean()
    data = DataLoader(ticker).get_data('2021-01-01', '2021-1-31')
    cap_gain = back_test(data, pep)
    dic[ticker] = {'PEP': pep, 'Above Pep': above_pep, 'Cap Gain': cap_gain}

/tmp/ipykernel_32881/3165097499.py:16: RuntimeWarning: invalid value encountered in double_scalars
  shares = capital // base


In [29]:
df = pd.DataFrame.from_dict(dic, orient='index')

In [36]:
mask = (df['Above Pep'] > .9) & (df['Cap Gain'] > .5)
df[mask].sort_values(by='Cap Gain', ascending=False)

,PEP,Above Pep,Cap Gain
KLDO,0.004960,0.907527,1.35718
ALDX,0.003761,0.905960,0.78892
CMRX,0.002777,0.915232,0.65084
PSNL,0.004540,0.901809,0.61106
MBIO,0.004122,0.903311,0.50063


In [42]:
for ticker in df[mask].index:
    data = DataLoader(ticker).get_data('2021-01-01', '2021-12-31')
    pep = df.loc[ticker, 'PEP']
    cap_gain = back_test(data, pep)
    print('{:>6} | {:>8.4f} | {:>8.4f}'.format(ticker, pep, cap_gain))

  ALDX |   0.0038 |   3.5078
  CMRX |   0.0028 |   4.4536
  KLDO |   0.0050 |  15.2914
  MBIO |   0.0041 |   3.2978
  PSNL |   0.0045 |   4.1264
